In [ ]:
# Define a function to create an animation based on a dataset and a start year
def scatter_animation(df, start_year, title=''):

    # Filter monthly data only and convert to datetime.
    df = df[df['Periods'].str.contains('MM')]
    df['Periods']= pd.to_datetime(df['Periods'], format='%YMM%m')

    # Select a start year
    df = df[df['Periods'].dt.year >= start_year]

    # Convert DataFrame to long form data for use with Plotly Express
    df = df.melt(id_vars=['Airports', 'Periods'])
    df = df[df['Airports'] == 'TOTAL']

    # Load Geometry
    geo_df = gpd.read_file('map_v2.geojson')

    # Create a list of geometry objects to assign to the data:
    loclist = []
    for i in range(len(df)):
        region = df.iloc[i, 2]
        location = geo_df[geo_df['id'] == region].iloc[0, 1]
        loclist.append(location)

    # Reset the index of df to ensure index of loclist matches dataset, and convert to a GeoDataframe
    df = df.reset_index()
    df['geometry'] = loclist
    gdf = gpd.GeoDataFrame(df)

    # Create the animation:
    fig = px.scatter_geo(gdf,
                        lat=gdf.geometry.y,
                        lon=gdf.geometry.x,
                        hover_name='variable',
                        size='value',
                        animation_frame='Periods',
                        color='variable',
                        title=title,
                        labels={'variable': 'Origin/Destination'})
    fig.show()


# Selecting and cleaning PASSENGER data
# Select the relevant columns for regional data.
passengers = data[['Airports', 'Periods', 'EUCountries_23', 'OtherEurope_24','NorthAfrica_27', 'WestAfrica_28', 'CentralAfrica_29', 'EastAfrica_30', 'SouthAfrica_31', 'NorthAmerica_33', 'CentralAmerica_34', 'SouthAmerica_35', 'WestAsia_37', 'SouthEastAsia_38', 'NorthEastAsia_39', 'Oceania_40']]

# Clean up the names of the columns
passengers = passengers.rename(columns = {'EUCountries_23': 'EU Countries'})

keys = list(passengers.keys()[3:])

for key in keys:
    wordlist = re.findall('[A-Z][^A-Z]*', key.split('_')[0])
    new_key = " ".join(wordlist)
    passengers = passengers.rename(columns = {key: new_key})

# Selecting and cleaning CARGO data
# Select the relevant columns for regional data.
cargo = data[['Airports', 'Periods', 'EUCountries_54', 'OtherEurope_55','NorthAfrica_58', 'WestAfrica_59', 'CentralAfrica_60', 'EastAfrica_61', 'SouthAfrica_62', 'NorthAmerica_64', 'CentralAmerica_65', 'SouthAmerica_66', 'WestAsia_68', 'SouthEastAsia_69', 'NorthEastAsia_70', 'Oceania_71']]

# Clean up the names of the columns
cargo = cargo.rename(columns = {'EUCountries_54': 'EU Countries'})

keys = list(cargo.keys()[3:])

for key in keys:
    wordlist = re.findall('[A-Z][^A-Z]*', key.split('_')[0])
    new_key = " ".join(wordlist)
    cargo = cargo.rename(columns = {key: new_key})

# Plot both datasets
scatter_animation(passengers, 2019, title='Passenger Volumes from Dutch Airports')
scatter_animation(cargo, 2019, title='Cargo Volumes from Dutch Airports')